<a href="https://colab.research.google.com/github/duongquangvinh/Fundamental-Machine-Learning-model/blob/main/keras_model_subclassing_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization, Activation

from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
import os

In [2]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

# scale the data to the range [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# Convert class vectors to binary class matrices.
num_classes = 10
trainY = to_categorical(trainY, num_classes)
testY = to_categorical(testY, num_classes)

# Data normalization
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255
testX /= 255

# Data augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [3]:
class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = Conv2D(32, (5,5), padding="same")
        self.act1 = Activation("relu")
        self.pool1 = MaxPooling2D(pool_size=(2, 2))
        self.conv2 = Conv2D(48, (5,5), padding="valid")
        self.act2 = Activation("relu")
        self.pool2 = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(256)
        self.act3 = Activation("relu")
        self.dense2 = Dense(84)
        self.act4 = Activation("relu")
        self.dense3 = Dense(10)
        self.act5 = Activation("softmax")
    def call(self, inputs):
        x = self.pool1(self.act1(self.conv1(inputs)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = self.flatten(x)
        x = self.act3(self.dense1(x))
        x = self.act4(self.dense2(x))
        x = self.act5(self.dense3(x))
        return x

In [4]:
net = LeNet()
net.compile(optimizer=Adam(learning_rate=5e-4),
                    loss=categorical_crossentropy,
                    metrics=['accuracy'])
history = net.fit(datagen.flow(trainX, trainY, batch_size=32),
                  steps_per_epoch=len(trainX)/32,
	            validation_data=(testX, testY),
	            epochs=5, verbose=1)

checkpoint_dir = "./checkpoint/"
os.makedirs(checkpoint_dir, exist_ok=True)

net.save_weights(os.path.join(checkpoint_dir, "keras_cifar10.h5"))

# evaluate model and save
_, acc = net.evaluate(testX, testY, verbose=0)
print(f"Accuracy: {acc*100}")

Epoch 1/5
1562/1562 [==============================] - 37s 23ms/step - loss: 2.0843 - accuracy: 0.2252 - val_loss: 1.9621 - val_accuracy: 0.2782
Epoch 2/5
1562/1562 [==============================] - 35s 22ms/step - loss: 1.9536 - accuracy: 0.2829 - val_loss: 1.8739 - val_accuracy: 0.3175
Epoch 3/5
1562/1562 [==============================] - 36s 23ms/step - loss: 1.8481 - accuracy: 0.3233 - val_loss: 1.7569 - val_accuracy: 0.3644
Epoch 4/5
1562/1562 [==============================] - 36s 23ms/step - loss: 1.7509 - accuracy: 0.3661 - val_loss: 1.6472 - val_accuracy: 0.4157
Epoch 5/5
1562/1562 [==============================] - 36s 23ms/step - loss: 1.6838 - accuracy: 0.3930 - val_loss: 1.5897 - val_accuracy: 0.4351
Accuracy: 43.50999891757965
